In [1]:
# imports
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import datetime, timedelta

/Users/MilesChild/opt/miniconda3/lib/python3.9/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


# Read Voting History

In [2]:
# read in the voting history data
votes = pd.read_excel('voting_history.xlsx')
votes['voting_date'] = votes['voting_date'].astype('str')
votes.head()

,voting_date,ticker,decision,yes,no
0,2024-04-14,PL,LONG,24,6
1,2024-04-10,PGY,SHORT,13,14
2,2024-04-10,CREX,LONG,26,2
3,2024-09-10,CLBT,SHORT,21,1
4,2024-04-14,TOST,LONG,28,3


# Obtain Historical Price Data

In [3]:
# Add in 1mo, 6mo, 1yr return columns to the votes dataframe
votes_with_returns = votes.copy()
votes_with_returns['1mo_return_asset'] = None
votes_with_returns['1mo_return_benchmark'] = None
votes_with_returns['6mo_return_asset'] = None
votes_with_returns['6mo_return_benchmark'] = None
votes_with_returns['1yr_return_asset'] = None
votes_with_returns['1yr_return_benchmark'] = None

In [4]:
# get end date for start date
def get_end_date(start_date: str, no_days: int):
    end_date = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=no_days)
    end_date = end_date.date()
    end_date = str(end_date)
    return end_date

# Method for getting stock data for a date range from yfinance
def __get_stock_data(ticker: str, start_date: str, end_date: str):
    # make sure start and end date are not in the future
    if datetime.strptime(start_date, '%Y-%m-%d') > datetime.now():
        raise ValueError("Start date is in the future")
    if datetime.strptime(end_date, '%Y-%m-%d') > datetime.now():
        raise ValueError("End date is in the future")
    
    data = yf.download(ticker, start_date, end_date)
    return data

def get_asset_returns(ticker: str, start_date: str, end_date: str):
    data = __get_stock_data(ticker, start_date, end_date)
    return_for_period = data['Adj Close'][-1] / data['Adj Close'][0] - 1
    return return_for_period

# Assertions 

assert get_end_date('2024-01-01', 30) == '2024-01-31'
# assert that this returns an error
try:
    __get_stock_data('TSLA', '2025-01-01', '2025-12-31')
except ValueError as e:
    i = 0

**Benchmark Returns**

In [5]:
# Fill in benchmark return columns
benchmark = '^RUT'

for index, row in votes_with_returns.iterrows():
    start = row['voting_date']
    one_mo_after = get_end_date(start, 30)
    six_mo_after = get_end_date(start, 180)
    one_yr_after = get_end_date(start, 365)
    try:
        one_mo_ret_benchmark = get_asset_returns(benchmark, start, one_mo_after)
    except ValueError as e:
        one_mo_ret_benchmark = None

    try:
        six_mo_ret_benchmark = get_asset_returns(benchmark, start, six_mo_after)
    except ValueError as e:
        six_mo_ret_benchmark = None
        
    try:
        one_yr_ret_benchmark = get_asset_returns(benchmark, start, one_yr_after)
    except ValueError as e:
        one_yr_ret_benchmark = None

    votes_with_returns.at[index, '1mo_return_benchmark'] = one_mo_ret_benchmark
    votes_with_returns.at[index, '6mo_return_benchmark'] = six_mo_ret_benchmark
    votes_with_returns.at[index, '1yr_return_benchmark'] = one_yr_ret_benchmark

votes_with_returns.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,voting_date,ticker,decision,yes,no,1mo_return_asset,1mo_return_benchmark,6mo_return_asset,6mo_return_benchmark,1yr_return_asset,1yr_return_benchmark
0,2024-04-14,PL,LONG,24,6,None,0.043736,None,None,None,None
1,2024-04-10,PGY,SHORT,13,14,None,0.022303,None,None,None,None
2,2024-04-10,CREX,LONG,26,2,None,0.022303,None,None,None,None
3,2024-09-10,CLBT,SHORT,21,1,None,None,None,None,None,None
4,2024-04-14,TOST,LONG,28,3,None,0.043736,None,None,None,None


**Asset Returns**

In [20]:
# Get the asset returns

for index, row in votes_with_returns.iterrows():
    ticker = row['ticker']
    start = row['voting_date']
    side = row['decision']
    one_mo_after = get_end_date(start, 30)
    six_mo_after = get_end_date(start, 180)
    one_yr_after = get_end_date(start, 365)
    try:
        try:
            one_mo_ret = get_asset_returns(ticker, start, one_mo_after)
            if side == 'SHORT':
                one_mo_ret = -one_mo_ret
        except ValueError as e:
            one_mo_ret = None

        try:
            six_mo_ret = get_asset_returns(ticker, start, six_mo_after)
            if side == 'SHORT':
                six_mo_ret = -six_mo_ret
        except ValueError as e:
            six_mo_ret = None
            
        try:
            one_yr_ret = get_asset_returns(ticker, start, one_yr_after)
            if side == 'SHORT':
                one_yr_ret = -one_yr_ret
        except ValueError as e:
            one_yr_ret = None
    except Exception as e:
        print(f'Skipping {ticker} due to error: {e}')
        continue

    votes_with_returns.at[index, '1mo_return_asset'] = one_mo_ret
    votes_with_returns.at[index, '6mo_return_asset'] = six_mo_ret
    votes_with_returns.at[index, '1yr_return_asset'] = one_yr_ret

votes_with_returns.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['KRTX']: Exce

Skipping KRTX due to error: index -1 is out of bounds for axis 0 with size 0


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Skipping MAXR due to error: index -1 is out of bounds for axis 0 with size 0


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,voting_date,ticker,decision,yes,no,1mo_return_asset,1mo_return_benchmark,6mo_return_asset,6mo_return_benchmark,1yr_return_asset,1yr_return_benchmark,1mo_alpha,6mo_alpha,1yr_alpha
0,2024-04-14,PL,LONG,24,6,0.088398,0.043736,None,None,None,None,0.044662,NaN,NaN
1,2024-04-10,PGY,SHORT,13,14,-0.146497,0.022303,None,None,None,None,0.124194,NaN,NaN
2,2024-04-10,CREX,LONG,26,2,0.238532,0.022303,None,None,None,None,0.216229,NaN,NaN
3,2024-09-10,CLBT,SHORT,21,1,None,None,None,None,None,None,NaN,NaN,NaN
4,2024-04-14,TOST,LONG,28,3,0.200889,0.043736,None,None,None,None,0.157153,NaN,NaN


In [21]:
votes_with_returns.tail()

,voting_date,ticker,decision,yes,no,1mo_return_asset,1mo_return_benchmark,6mo_return_asset,6mo_return_benchmark,1yr_return_asset,1yr_return_benchmark,1mo_alpha,6mo_alpha,1yr_alpha
37,2020-12-12,EVVTY,LONG,59,27,0.025692,0.092901,1.007361,0.215935,0.293376,0.15568,-0.06721,0.791426,0.137696
38,2020-12-12,REGI,LONG,83,1,0.30017,0.092901,0.097177,0.215935,-0.302329,0.15568,0.207268,-0.118758,-0.458009
39,2020-12-12,EAF,SHORT,36,46,-0.20829,0.092901,-0.322241,0.215935,-0.241334,0.15568,0.115389,0.106305,0.085654
40,2020-12-12,INVH,LONG,40,33,0.01911,0.092901,0.315026,0.215935,0.513116,0.15568,-0.073791,0.099091,0.357435
41,2020-12-12,GLXZ,LONG,65,21,0.090909,0.092901,1.181818,0.215935,1.242424,0.15568,-0.001992,0.965883,1.086744


# Calculate Alphas

alpha = true if N-period return of equity > N-period return of benchmark else false

In [23]:
votes_with_returns['1mo_alpha'] = votes_with_returns['1mo_return_asset'] - votes_with_returns['1mo_return_benchmark']
votes_with_returns['6mo_alpha'] = votes_with_returns['6mo_return_asset'] - votes_with_returns['6mo_return_benchmark']
votes_with_returns['1yr_alpha'] = votes_with_returns['1yr_return_asset'] - votes_with_returns['1yr_return_benchmark']
votes_with_returns.tail()

,voting_date,ticker,decision,yes,no,1mo_return_asset,1mo_return_benchmark,6mo_return_asset,6mo_return_benchmark,1yr_return_asset,1yr_return_benchmark,1mo_alpha,6mo_alpha,1yr_alpha
37,2020-12-12,EVVTY,LONG,59,27,0.025692,0.092901,1.007361,0.215935,0.293376,0.15568,-0.06721,0.791426,0.137696
38,2020-12-12,REGI,LONG,83,1,0.30017,0.092901,0.097177,0.215935,-0.302329,0.15568,0.207268,-0.118758,-0.458009
39,2020-12-12,EAF,SHORT,36,46,-0.20829,0.092901,-0.322241,0.215935,-0.241334,0.15568,-0.301191,-0.538176,-0.397014
40,2020-12-12,INVH,LONG,40,33,0.01911,0.092901,0.315026,0.215935,0.513116,0.15568,-0.073791,0.099091,0.357436
41,2020-12-12,GLXZ,LONG,65,21,0.090909,0.092901,1.181818,0.215935,1.242424,0.15568,-0.001992,0.965883,1.086744


In [29]:
# Histogram the 1mo alpha for all dates before 2022
print(votes_with_returns['1yr_alpha'].mean())
plt = go.Histogram(x=votes_with_returns['1yr_alpha'], nbinsx=100)
fig = go.Figure(plt)
fig.show()

0.08183230516122474


In [33]:
# Get the percent of time the alpha is positive on each time scale 
print(f'1mo alpha is positive {votes_with_returns[votes_with_returns["1mo_alpha"] > 0].shape[0] / votes_with_returns.shape[0] * 100}% of the time')
print(f'6mo alpha is positive {votes_with_returns[votes_with_returns["6mo_alpha"] > 0].shape[0] / votes_with_returns.shape[0] * 100}% of the time')
print(f'1yr alpha is positive {votes_with_returns[votes_with_returns["1yr_alpha"] > 0].shape[0] / votes_with_returns.shape[0] * 100}% of the time')

1mo alpha is positive 47.61904761904761% of the time
6mo alpha is positive 50.0% of the time
1yr alpha is positive 40.476190476190474% of the time
